# BÁO CÁO ĐÁNH GIÁ DỮ LIỆU HUẤN LUYỆN VÀ TEST SET

# Hội thi Thử thách Trí tuệ Nhân tạo Thành phố Hồ Chí Minh 2024

## Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2024. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

## Cài đặt ban đầu

Bạn cần cài đặt môi trường để chạy được notebook này trên máy tính cá nhân của bạn. Hướng dẫn này không bao gồm phần cài đặt môi trường. Khuyến nghị: các bạn có thể cài đặt [Anaconda](https://docs.anaconda.com/free/anaconda/install/windows/).

## Cài đặt các thư viện FiftyOne và PyTorch
Hướng dẫn này dùng fiftyone là công cụ để trực quan dữ liệu và pytorch là backend chính cho các thuật toán máy học.


In [1]:
! pip install fiftyone==0.24.1
! pip install torch torchvision torchaudio


Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [2]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os



Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

In [6]:
dataset = fo.Dataset.from_images_dir('/workspaces/AIC/keyframes', name=None, tags=None, recursive=True)

 100% |███████████████| 8115/8115 [725.1ms elapsed, 0s remaining, 11.2K samples/s]      


Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5151](http://localhost:5151)

In [7]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


Hoặc bạn có thể chạy cell bên dưới để mở tab mới cho ứng dụng web fiftyone

In [8]:
session.open_tab()

<IPython.core.display.Javascript object>

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [9]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('/', 2)
    sample.save()

Bạn có thể xem `Sample` đầu tiên của `Dataset` bằng lệnh sau:

In [10]:
print(dataset.first())

<Sample: {
    'id': '66dab0ec05658189ba43a9da',
    'media_type': 'image',
    'filepath': '/workspaces/AIC/keyframes/L01_V001/001.jpg',
    'tags': [],
    'metadata': None,
    'video': 'L01_V001',
    'frameid': '001',
}>


### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [66]:
for sample in dataset:
    object_path = f"D:\\AICBaseline\\objects-sample\\objects\\{sample['video']}\\{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()


### Thêm thông tin CLIP embedding.

In [47]:
all_keyframe = glob('/workspaces/course-master-of-AI-/dataset/AIC/keyframe/*/*.jpg')
video_keyframe_dict = {}
all_video = glob('/workspaces/course-master-of-AI-/dataset/AIC/keyframe/*')
all_video = [v.rsplit('/',1)[-1] for v in all_video]

In [48]:
all_keyframe

[]

In [46]:
all_video

[]

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [37]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('/',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [38]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [39]:
for v in all_video:
    print(v)

L01_V001.mp4
L01_V002.mp4
L01_V003.mp4
L01_V004.mp4
L01_V005.mp4
L01_V006.mp4
L01_V007.mp4
L01_V008.mp4
L01_V009.mp4
L01_V010.mp4
L01_V011.mp4
L01_V012.mp4
L01_V013.mp4
L01_V014.mp4
L01_V015.mp4
L01_V016.mp4
L01_V017.mp4
L01_V018.mp4
L01_V019.mp4
L01_V020.mp4
L01_V021.mp4
L01_V022.mp4
L01_V023.mp4
L01_V024.mp4
L01_V025.mp4
L01_V026.mp4
L01_V027.mp4
L01_V028.mp4
L01_V029.mp4
L01_V030.mp4
L01_V031.mp4


In [40]:
embedding_dict = {}
for v in all_video:
    clip_path = f'/workspaces/course-master-of-AI-/dataset/AIC/clip-features-vit-b32-sample/clip-features/{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]


FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/course-master-of-AI-/dataset/AIC/clip-features-vit-b32-sample/clip-features/L01_V001.mp4.npy'

Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [30]:
embedding_dict

{}

In [ ]:
for sample in dataset:
    print(sample['video'])
    print(sample['frameid'])

In [28]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


KeyError: 'L01_V001'

In [93]:
embedding_dict.keys()

dict_keys(['L01_V001', 'L01_V002', 'L01_V003', 'L01_V004', 'L01_V005', 'L01_V006', 'L01_V007', 'L01_V008', 'L01_V009', 'L01_V010', 'L01_V011', 'L01_V012', 'L01_V013', 'L01_V014', 'L01_V015', 'L01_V016', 'L01_V017', 'L01_V018', 'L01_V019', 'L01_V020', 'L01_V021', 'L01_V022', 'L01_V023', 'L01_V024', 'L01_V025', 'L01_V026', 'L01_V027', 'L01_V028', 'L01_V029', 'L01_V030', 'L01_V031'])

In [94]:
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="img_sim",
)

## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [ ]:
# Bạn cần phải cài version umap-learn hỗ trợ.
# fob.compute_visualization(
#     dataset,
#     embeddings=clip_embeddings,
#     brain_key="img_viz"
# )
